# 1. 기본 설정 및 유틸리티 함수


In [ ]:
import requests
import pandas as pd
import json
import time
from google.colab import userdata
from datetime import datetime
import os

# [재사용] 에러 파싱 함수
def parse_api_error(e: Exception) -> str:
    error_message = str(e)
    if '?serviceKey=' in error_message:
        parsed_message = error_message.split('?serviceKey=')[0] + "?serviceKey=... [KEY HIDDEN]"
        return parsed_message
    return error_message

# [경로 설정]
SAVE_FOLDER_PATH = "/content/drive/MyDrive/D-up_Fest_compass/중간발표 코랩/data"

In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')
os.makedirs(SAVE_FOLDER_PATH, exist_ok=True)

print(f"✅ 기본 설정 및 에러 파싱 함수가 로드되었습니다.")
print(f"✅ 데이터 저장 경로: {SAVE_FOLDER_PATH}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 기본 설정 및 에러 파싱 함수가 로드되었습니다.
✅ 데이터 저장 경로: /content/drive/MyDrive/D-up_Fest_compass/중간발표 코랩/data


# 2. API 데이터 수집 함수 (질적+양적 통합)

In [ ]:
def fetch_festival_data_by_period(service_url: str, app_name: str, start_date: str, end_date: str) -> list:
    """
    지정된 'searchFestival' API를 호출하여
    특정 기간 동안 서울(areaCode=1)의 모든 데이터를 수집합니다.
    (페이지네이션 로직 포함)
    """
    try:
        SERVICE_KEY = userdata.get('Data_go_kr_API_key')
        if not SERVICE_KEY:
            raise Exception("Colab '비밀'에서 'Data_go_kr_API_key'를 찾을 수 없습니다.")
    except Exception as e:
        print(f"❌ [에러] Colab '비밀' 로딩 실패: {e}")
        return []

    all_items = []
    page_no = 1
    total_count = -1 # (총 개수를 모르므로 -1에서 시작)
    num_of_rows = 100 # (한 번에 100개씩 요청)

    while True:
        params = {
            "serviceKey": SERVICE_KEY, "MobileOS": "ETC", "MobileApp": app_name,
            "eventStartDate": start_date,
            "eventEndDate": end_date,
            "areaCode": "1", # 서울
            "numOfRows": num_of_rows,
            "pageNo": page_no,
            "_type": "json"
        }

        try:
            response = requests.get(service_url, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()

            # [수정] 비표준/표준 에러 공통 처리
            if 'response' not in data or data['response']['header']['resultCode'] != '0000':
                msg = "Unknown Error"
                if 'response' in data:
                    msg = data['response']['header']['resultMsg']
                elif 'resultMsg' in data:
                    msg = data['resultMsg']
                print(f"[{app_name}] Page {page_no} API Error: {msg}")
                break # 에러 발생 시 반복 중단

            body = data['response']['body']

            # 첫 번째 호출(page_no==1)일 때만 totalCount 설정
            if page_no == 1:
                total_count = body.get('totalCount', 0)
                if total_count == 0:
                    print(f"[{app_name}] 수집할 데이터가 0건입니다.")
                    break
                print(f"[{app_name}] 총 {total_count}건의 데이터를 확인했습니다. 수집을 시작합니다...")

            # items가 비어있거나 item이 없는 경우
            if 'items' not in body or not body['items'] or 'item' not in body['items']:
                print(f"[{app_name}] Page {page_no}: 데이터가 없습니다. (수집 완료)")
                break

            items = body['items']['item']
            if isinstance(items, dict): # 데이터가 1개인 경우
                items = [items]

            all_items.extend(items)
            print(f"[{app_name}] Page {page_no}: {len(items)}건 수집 완료 (누적: {len(all_items)})")

            # [수정] 마지막 페이지 계산
            if len(all_items) >= total_count:
                print(f"[{app_name}] 모든 데이터 수집 완료.")
                break

            page_no += 1
            time.sleep(0.05) # API 부하 방지

        except requests.exceptions.HTTPError as e:
            print(f"[{app_name}] Page {page_no} HTTP Error: {parse_api_error(e)}")
            break
        except Exception as e:
            print(f"[{app_name}] Page {page_no} Unknown Error: {e}")
            break

    return all_items

print("✅ 'fetch_festival_data_by_period' (기간검색) 함수가 로드되었습니다.")

✅ 'fetch_festival_data_by_period' (기간검색) 함수가 로드되었습니다.


# 3. 데이터 수집 및 저장

### 3.1. 국문 데이터 수집 및 저장



In [ ]:
# 1. '국문' API 파라미터 정의
KOR_URL = "http://apis.data.go.kr/B551011/KorService2/searchFestival2"
KOR_APP_NAME = "ProjectA-Kor-Period"

# 2. 수집 기간 정의 (빅데이터와 동일한 6월~8월)
start_date_str = "20250101"
end_date_str = "20251231"

print(f"--- [3-A] '국문' 행사 목록 수집 시작 (기간: {start_date_str} ~ {end_date_str}) ---")
# 셀 2에서 정의한 범용 함수 호출
korean_festivals_list = fetch_festival_data_by_period(KOR_URL, KOR_APP_NAME, start_date_str, end_date_str)

# 3. 파일로 저장
if korean_festivals_list:
    df_kor = pd.DataFrame(korean_festivals_list)
    save_path = os.path.join(SAVE_FOLDER_PATH, "seoul_festivals_kor.csv")
    df_kor.to_csv(save_path, index=False, encoding='utf-8-sig')

    print(f"✅ [국문] 성공: {save_path}에 총 {len(df_kor)}건 저장 완료.")
    print(df_kor[['contentid', 'title', 'sigungucode']].head())
else:
    print("❌ [국문] 수집된 데이터가 없습니다.")

--- [3-A] '국문' 행사 목록 수집 시작 (기간: 20250101 ~ 20251231) ---
[ProjectA-Kor-Period] 총 239건의 데이터를 확인했습니다. 수집을 시작합니다...
[ProjectA-Kor-Period] Page 1: 100건 수집 완료 (누적: 100)
[ProjectA-Kor-Period] Page 2: 100건 수집 완료 (누적: 200)
[ProjectA-Kor-Period] Page 3: 39건 수집 완료 (누적: 239)
[ProjectA-Kor-Period] 모든 데이터 수집 완료.
✅ [국문] 성공: /content/drive/MyDrive/D-up_Fest_compass/중간발표 코랩/data/seoul_festivals_kor.csv에 총 239건 저장 완료.
  contentid           title sigungucode
0   3113671  가락몰 빵축제 전국빵지자랑          18
1   3379778   가락옥토버페스트 미식야행          18
2   3021762       가을 , 명동으로          24
3   3541003          강남청년축제           1
4   1806376         강동북페스티벌           2


In [ ]:
df_kor

,addr1,addr2,zipcode,cat1,cat2,cat3,contentid,contenttypeid,createdtime,eventstartdate,...,sigungucode,tel,title,lDongRegnCd,lDongSignguCd,lclsSystm1,lclsSystm2,lclsSystm3,progresstype,festivaltype
0,서울특별시 송파구 양재대로 932 (가락동),가락몰 3층 하늘공원,05699,A02,A0207,A02070200,3113671,15,20240415100726,20250509,...,18,02-3435-0286,가락몰 빵축제 전국빵지자랑,11,710,EV,EV01,EV010600,,
1,서울특별시 송파구 양재대로 932 (가락동),(가락동 600) 가락몰3층 하늘공원,05699,A02,A0207,A02070200,3379778,15,20241004095015,20251024,...,18,02-3435-0455,가락옥토버페스트 미식야행,11,710,EV,EV01,EV010600,,
2,서울특별시 중구 퇴계로 지하126 (충무로2가),,04629,A02,A0208,A02081300,3021762,15,20231020110542,20251115,...,24,02-2277-8971,"가을 , 명동으로",11,140,EV,EV03,EV030400,선택안함,선택안함
3,서울특별시 강남구 도산대로45길 20 (신사동),도산공원,06020,A02,A0207,A02070200,3541003,15,20250918175214,20250919,...,1,010-9771-2090,강남청년축제,11,680,EV,EV01,EV010600,,
4,서울특별시 강동구 양재대로84길 63 (둔촌동),강동중앙도서관·둔촌 1동 근린공원,05411,A02,A0207,A02070200,1806376,15,20130422192127,20251101,...,2,02-2045-7919,강동북페스티벌,11,740,EV,EV01,EV010200,선택안함,선택안함
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,서울특별시 강서구 마곡중앙로 143 (마곡동),,07797,A02,A0208,A02080600,3369559,15,20240919155523,20250829,...,4,02-761-2514~5,K-푸드페스타,11,500,EV,EV03,EV030200,선택안함,선택안함
235,서울특별시 강남구 영동대로 513,(삼성동),06164,A02,A0208,A02081300,2381938,15,20160503022604,20251218,...,1,02-761-2512,K-핸드메이드페어 2025,11,680,EV,EV03,EV030400,선택안함,선택안함
236,서울특별시 성동구 자동차시장길 49 (용답동),서울새활용플라자,04807,A02,A0207,A02070200,3486606,15,20250411180007,20250418,...,16,02-2153-0421,"My Own Planet, 지구야 놀자",11,200,EV,EV01,EV010200,,
237,서울특별시 송파구 올림픽로 424 (방이동),,05540,A02,A0208,A02081300,3488940,15,20250430092032,20250530,...,18,070-4288-4097,MyK FESTA(마이케이 페스타),11,710,EV,EV03,EV030400,선택안함,선택안함


### 3.2. 영문 데이터 수집 및 저장

In [ ]:
# 1. '영문' API 파라미터 정의
ENG_URL = "http://apis.data.go.kr/B551011/EngService2/searchFestival2" # 👈 EngService2/searchFestival2
ENG_APP_NAME = "ProjectA-Eng-Period"

# 2. 수집 기간 정의 (국문과 동일)
start_date_str = "20250101"
end_date_str = "20251231"

print(f"\n--- [3-B] '영문' 행사 목록 수집 시작 (기간: {start_date_str} ~ {end_date_str}) ---")
# 셀 2에서 정의한 범용 함수 호출
english_festivals_list = fetch_festival_data_by_period(ENG_URL, ENG_APP_NAME, start_date_str, end_date_str)

# 3. 파일로 저장
if english_festivals_list:
    df_eng = pd.DataFrame(english_festivals_list)
    save_path = os.path.join(SAVE_FOLDER_PATH, "seoul_festivals_eng.csv")
    df_eng.to_csv(save_path, index=False, encoding='utf-8-sig')

    print(f"✅ [영문] 성공: {save_path}에 총 {len(df_eng)}건 저장 완료.")
    print(df_eng[['contentid', 'title', 'sigungucode']].head())
else:
    print("❌ [영문] 수집된 데이터가 없습니다.")


--- [3-B] '영문' 행사 목록 수집 시작 (기간: 20250101 ~ 20251231) ---
[ProjectA-Eng-Period] 총 58건의 데이터를 확인했습니다. 수집을 시작합니다...
[ProjectA-Eng-Period] Page 1: 58건 수집 완료 (누적: 58)
[ProjectA-Eng-Period] 모든 데이터 수집 완료.
✅ [영문] 성공: /content/drive/MyDrive/D-up_Fest_compass/중간발표 코랩/data/seoul_festivals_eng.csv에 총 58건 저장 완료.
  contentid                                          title sigungucode
0   3544280  100 Years Market 100 Years Night (백년시장 백년나이트)           3
1   3500481             2025 APP WORLD TOUR SEOUL SUP OPEN          18
2   2456838          Achasan Sunrise Festival (아차산 해맞이 축제)           6
3   3523415                                   Almond (아몬드)          23
4   3529576                         Another Today(또 다른 오늘)          23


In [ ]:
df_eng

,addr1,addr2,zipcode,cat1,cat2,cat3,contentid,contenttypeid,createdtime,eventstartdate,...,modifiedtime,areacode,sigungucode,tel,title,lDongRegnCd,lDongSignguCd,lclsSystm1,lclsSystm2,lclsSystm3
0,"16 Hancheon-ro 144-gil, Gangbuk-gu, Seoul",,01054,A02,A0207,A02070200,3544280,85,20250925140512,20250926,...,20251023143000,1,3,+82-2-903-9110,100 Years Market 100 Years Night (백년시장 백년나이트),11,305,EV,EV01,EV010600
1,"65 Hangaram-ro, Songpa-gu, Seoul",,05502,A02,A0208,A02081300,3500481,85,20250624164230,20250705,...,20250625163713,1,18,+82-2-6677-1599,2025 APP WORLD TOUR SEOUL SUP OPEN,11,710,EV,EV03,EV030400
2,"Achasan, Gwangjin-gu, Seoul",,04965,A02,A0207,A02070200,2456838,85,20161215014035,20250101,...,20241219163944,1,6,+82-2-450-7596,Achasan Sunrise Festival (아차산 해맞이 축제),11,215,EV,EV01,EV010200
3,서울특별시 종로구 대학로12길 64 (동숭동),NOL Uniflex Theater,03086,A02,A0208,A02080300,3523415,85,20250819151536,20250919,...,20251023143000,1,23,,Almond (아몬드),11,110,EV,EV02,EV020300
4,서울특별시 종로구 동숭길 47 (동숭동),대학로 나인진홀 1관,03086,A02,A0208,A02080200,3529576,85,20250829154645,20251018,...,20250916094141,1,23,,Another Today(또 다른 오늘),11,110,EV,EV02,EV020200
5,서울특별시 중구 명동길 35 (명동1가),국립극단 명동예술극장,04534,A02,A0208,A02080200,3532653,85,20250908112749,20251010,...,20250916180627,1,24,,Anthropolis 1 : Prolog/Dionysos(안트로폴리스 1 : 프롤로...,11,140,EV,EV02,EV020200
6,"206 Songpanaru-gil, Songpa-gu, Seoul",,05617,A02,A0207,A02070200,3421970,85,20241119155027,20241025,...,20241121091157,1,18,+82-2-2147-2110,Autumn and Winter of the Lake and the Luminari...,11,710,EV,EV01,EV010500
7,서울특별시 종로구 종로1길 45 (세종로),의정부지 역사유적광장(광화문),03141,A02,A0208,A02080100,3379755,85,20241004092150,20251003,...,20251023143000,1,24,,Bongsan Mask Dance(봉산탈춤 다 모여라~ ),11,110,EV,EV02,EV020100
8,서울특별시 종로구 대학로10길 24,(동숭동)대학로 틴틴홀,03086,A02,A0208,A02080200,3352421,85,20240822171941,20100406,...,20251023143000,1,23,,Cat on the Rooftop(옥탑방 고양이),11,110,EV,EV02,EV020200
9,"99, Sejong-daero, Jung-gu, Seoul",,04519,A02,A0208,A02080100,292962,85,20060227192042,20250101,...,20250711141018,1,24,"• 1330 Travel Hotline: +82-2-1330<br>(Korean, ...",Deoksugung Palace Royal Guard Changing Ceremon...,11,140,EV,EV02,EV020100


# 축제에 대한 상세정보 수집

## 국문 행사 상세정보 수집

In [ ]:
import requests
import pandas as pd
import time
from google.colab import userdata
import json
from tqdm.auto import tqdm

print("--- [Step 3-C] '국문' 행사 상세설명(overview) 수집 (최종 수정본) ---")

# --- 1. [입력 파일] 3-A에서 생성한 국문 축제 목록 로드 ---
PERIOD = "202501-202512" # 3-A 수집 기간과 일치
BASE_PATH = "/content/drive/MyDrive/D-up_Fest_compass/중간발표 코랩/data/"
INPUT_KOR_CSV_PATH = f"{BASE_PATH}seoul_festivals_kor.csv"
OUTPUT_KOR_CSV_PATH = f"{BASE_PATH}seoul_festivals_kor_detail_{PERIOD}.csv"

try:
    df_festivals_kor = pd.read_csv(INPUT_KOR_CSV_PATH)
    print(f"✅ '국문' 목록 파일({len(df_festivals_kor)}건) 로드 성공.")
except FileNotFoundError:
    print(f"❌ [에러] 입력 파일 '{INPUT_KOR_CSV_PATH}'를 찾을 수 없습니다.")
    raise FileNotFoundError

# --- 2. '국문' 상세 API 호출 함수 (최종 수정) ---
def get_korean_detail(content_id: str) -> str:
    KOR_DETAIL_URL = "http://apis.data.go.kr/B551011/KorService2/detailCommon2"

    try:
        # [수정 1] Colab Secrets 변수명을 'Data_go_kr_API_key'로 통일 (dom_for_TourAPI_collect.ipynb 셀 4와 일치시킴)
        SERVICE_KEY = userdata.get('Data_go_kr_API_key')
        if not SERVICE_KEY:
            print(f"  (Warn) contentid {content_id} - Secrets 'Data_go_kr_API_key'를 찾을 수 없습니다.")
            return None

    except Exception as e:
        print(f"  (Warn) contentid {content_id} - Secrets 로드 실패: {e}")
        return None

    params = {
        "serviceKey": SERVICE_KEY,
        "MobileOS": "ETC",
        "MobileApp": "ProjectA-KorDetail",
        "contentId": content_id,
        "_type": "json"
    }

    try:
        response = requests.get(KOR_DETAIL_URL, params=params, timeout=5)
        response.raise_for_status()
        data = response.json()

        # [디버깅용] 성공 시 원본 응답 확인 (필요시 주석 해제)
        # print(f"  [디버그] contentid {content_id} 원본 응답: {data}")

        # [수정 3] 'response' 키가 있는지 먼저 확인
        if 'response' not in data or 'header' not in data.get('response', {}):
            print(f"  (!!!!!!) contentid {content_id} - 심각: API 응답 구조 오류. 원본: {data}")
            return None

        # 이제 data['response']는 안전하게 접근 가능
        if data['response']['header']['resultCode'] == '0000':
            item_list = data['response']['body']['items'].get('item')

            if item_list and isinstance(item_list, list):
                item_object = item_list[0]
                return item_object.get('overview', None) # 정상 추출
            elif item_list and isinstance(item_list, dict):
                 return item_list.get('overview', None) # 아이템이 1개일 경우
            else:
                print(f"  (Info) contentid {content_id} - 'item' 리스트가 비어있음")
                return None
        else:
            # API 키 인증 실패, 트래픽 초과 등
            print(f"  (Warn) contentid {content_id} API 에러: {data['response']['header']['resultMsg']}")
            return None

    except requests.exceptions.HTTPError as http_err:
        # 404, 500, 503 등 HTTP 에러
        print(f"  (Warn) contentid {content_id} HTTP 에러: {http_err}")
        return None
    except Exception as e:
        # JSON 파싱 실패 등 기타 에러
        print(f"  (Warn) contentid {content_id} 처리 중 알 수 없는 에러: {e}")
        return None

# --- 3. 58건 반복 호출 및 저장 ---
detail_results = []
if 'df_festivals_kor' in locals() and not df_festivals_kor.empty:
    print(f"총 {len(df_festivals_kor)}건의 국문 'overview' 수집을 시작합니다...")

    for index, row in tqdm(df_festivals_kor.iterrows(), total=len(df_festivals_kor)):
        content_id = row['contentid']
        overview_text = get_korean_detail(content_id)

        detail_results.append({
            "contentid": content_id,
            "title": row['title'],
            "sigungucode": row['sigungucode'],
            "overview": overview_text
        })
        # time.sleep(0.1)

    # --- 4. 최종 파일로 저장 ---
    df_kor_detail = pd.DataFrame(detail_results)
    df_kor_detail.to_csv(OUTPUT_KOR_CSV_PATH, index=False, encoding='utf-8-sig')

    print("---" * 10)
    print(f"✅ [3-C 완료] {OUTPUT_KOR_CSV_PATH} 파일 저장 완료.")
    print(df_kor_detail.head().to_string(formatters={'overview': lambda x: str(x)[:50] + '...' if pd.notnull(x) else 'None'}))

--- [Step 3-C] '국문' 행사 상세설명(overview) 수집 (최종 수정본) ---
✅ '국문' 목록 파일(239건) 로드 성공.
총 239건의 국문 'overview' 수집을 시작합니다...


  0%|          | 0/239 [00:00<?, ?it/s]

------------------------------
✅ [3-C 완료] /content/drive/MyDrive/D-up_Fest_compass/중간발표 코랩/data/seoul_festivals_kor_detail_202501-202512.csv 파일 저장 완료.
   contentid           title  sigungucode                                               overview
0    3113671  가락몰 빵축제 전국빵지자랑           18  전국 각지의 농수축산물이 모이는 가락몰에서, 전국 각지의 빵 맛집들이 모여 서울 최초의 전...
1    3379778   가락옥토버페스트 미식야행           18  전국 각지 농수산물이 모이는 서울시 송파구 가락몰 3층 하늘공원에서 루프탑 야외축제 '제2...
2    3021762       가을 , 명동으로           24  '가을, 명동으로'는 즐거움과 의미가 함께하는 행사이다. 명동역 3번 출구 앞 ‘재미로 상...
3    3541003          강남청년축제            1  2025 강남청년축제 ‘소셜링 페스티벌’은 강남구를 대표하는 청년축제이다. 축제는 9월 1...
4    1806376         강동북페스티벌            2  강동북페스티벌은 강동문화재단이 주최하고 강동구립도서관이 주관하는 독서문화축제로서, 올해로 ...


In [ ]:
df_kor_detail

,contentid,title,sigungucode,overview
0,3113671,가락몰 빵축제 전국빵지자랑,18,"전국 각지의 농수축산물이 모이는 가락몰에서, 전국 각지의 빵 맛집들이 모여 서울 최..."
1,3379778,가락옥토버페스트 미식야행,18,전국 각지 농수산물이 모이는 서울시 송파구 가락몰 3층 하늘공원에서 루프탑 야외축제...
2,3021762,"가을 , 명동으로",24,"'가을, 명동으로'는 즐거움과 의미가 함께하는 행사이다. 명동역 3번 출구 앞 ‘재..."
3,3541003,강남청년축제,1,2025 강남청년축제 ‘소셜링 페스티벌’은 강남구를 대표하는 청년축제이다. 축제는 ...
4,1806376,강동북페스티벌,2,"강동북페스티벌은 강동문화재단이 주최하고 강동구립도서관이 주관하는 독서문화축제로서, ..."
...,...,...,...,...
234,3369559,K-푸드페스타,4,세상 모든 음식을 한 곳에서 만나볼 수 있는 ‘K-푸드페스타’가 8월 ‘서울 코엑스...
235,2381938,K-핸드메이드페어 2025,1,"‘K-핸드메이드페어’는 국내 최대 핸드메이드 페어로서, 아티스트와 소비자 간의 교류..."
236,3486606,"My Own Planet, 지구야 놀자",16,서울새활용플라자(SUP)가 2025년 4월 제55회 지구의 날(Earth Day)을...
237,3488940,MyK FESTA(마이케이 페스타),18,(MyK FESTA) MyK FESTA는 2025년 6월 19일부터 22일까지 4일...


## 영문 행사 상세정보 수집

In [ ]:
import requests
import pandas as pd
import time
from google.colab import userdata
import json
from tqdm.auto import tqdm

print("--- [Step 3-D] '영문' 행사 상세설명(overview) 수집 (최종 수정본) ---")

# --- 1. [입력 파일] 3-B에서 생성한 영문 축제 목록 로드 ---
PERIOD = "202501-202512"
BASE_PATH = "/content/drive/MyDrive/D-up_Fest_compass/중간발표 코랩/data/"
INPUT_ENG_CSV_PATH = f"{BASE_PATH}seoul_festivals_eng.csv"
OUTPUT_ENG_CSV_PATH = f"{BASE_PATH}seoul_festivals_eng_detail_{PERIOD}.csv"

try:
    df_festivals_eng = pd.read_csv(INPUT_ENG_CSV_PATH)
    print(f"✅ '영문' 목록 파일({len(df_festivals_eng)}건) 로드 성공.")
except FileNotFoundError:
    print(f"❌ [에러] 입력 파일 '{INPUT_ENG_CSV_PATH}'를 찾을 수 없습니다.")
    raise FileNotFoundError

# --- 2. '영문' 상세 API 호출 함수 (버그 수정됨) ---
def get_english_detail(content_id: str) -> str:
    ENG_DETAIL_URL = "http://apis.data.go.kr/B551011/EngService2/detailCommon2"

    try:
        # [수정 1] Colab Secrets 변수명을 'Data_go_kr_API_key'로 통일 (dom_for_TourAPI_collect.ipynb 셀 4와 일치시킴)
        SERVICE_KEY = userdata.get('Data_go_kr_API_key')
        if not SERVICE_KEY:
            print(f"  (Warn) contentid {content_id} - Secrets 'Data_go_kr_API_key'를 찾을 수 없습니다.")
            return None

    except Exception as e:
        print(f"  (Warn) contentid {content_id} - Secrets 로드 실패: {e}")
        return None

    params = {
        "serviceKey": SERVICE_KEY,
        "MobileOS": "ETC",
        "MobileApp": "ProjectA-EngDetail",
        "contentId": content_id,
        "_type": "json"
    }

    try:
        response = requests.get(ENG_DETAIL_URL, params=params, timeout=5)
        response.raise_for_status()
        data = response.json()

        # [디버깅용] 성공 시 원본 응답 확인 (필요시 주석 해제)
        # print(f"  [디버그] contentid {content_id} 원본 응답: {data}")

        # [수정 3] 'response' 키가 있는지 먼저 확인
        if 'response' not in data or 'header' not in data.get('response', {}):
            print(f"  (!!!!!!) contentid {content_id} - 심각: API 응답 구조 오류. 원본: {data}")
            return None

        # 이제 data['response']는 안전하게 접근 가능
        if data['response']['header']['resultCode'] == '0000':
            item_list = data['response']['body']['items'].get('item')

            if item_list and isinstance(item_list, list):
                item_object = item_list[0]
                return item_object.get('overview', None) # 정상 추출
            elif item_list and isinstance(item_list, dict):
                 return item_list.get('overview', None) # 아이템이 1개일 경우
            else:
                print(f"  (Info) contentid {content_id} - 'item' 리스트가 비어있음")
                return None
        else:
            # API 키 인증 실패, 트래픽 초과 등
            print(f"  (Warn) contentid {content_id} API 에러: {data['response']['header']['resultMsg']}")
            return None

    except requests.exceptions.HTTPError as http_err:
        # 404, 500, 503 등 HTTP 에러
        print(f"  (Warn) contentid {content_id} HTTP 에러: {http_err}")
        return None
    except Exception as e:
        # JSON 파싱 실패 등 기타 에러
        print(f"  (Warn) contentid {content_id} 처리 중 알 수 없는 에러: {e}")
        return None

# --- 3. 58건 반복 호출 및 저장 ---
detail_results = []
if 'df_festivals_eng' in locals() and not df_festivals_eng.empty:
    print(f"총 {len(df_festivals_eng)}건의 영문 'overview' 수집을 시작합니다...")

    for index, row in tqdm(df_festivals_eng.iterrows(), total=len(df_festivals_eng)):
        content_id = row['contentid']
        overview_text = get_english_detail(content_id)

        detail_results.append({
            "contentid": content_id,
            "title": row['title'],
            "sigungucode": row['sigungucode'],
            "overview": overview_text
        })
        # time.sleep(0.1)

    # --- 4. 최종 파일로 저장 ---
    df_eng_detail = pd.DataFrame(detail_results)
    df_eng_detail.to_csv(OUTPUT_ENG_CSV_PATH, index=False, encoding='utf-8-sig')

    print("---" * 10)
    print(f"✅ [3-D 완료] {OUTPUT_ENG_CSV_PATH} 파일 저장 완료.")
    print(df_eng_detail.head().to_string(formatters={'overview': lambda x: str(x)[:50] + '...' if pd.notnull(x) else 'None'}))

else:
    print("---" * 10)
    print("⚠️ [3-D 중단] 'df_festivals_eng' 변수가 로드되지 않았습니다.")

--- [Step 3-D] '영문' 행사 상세설명(overview) 수집 (최종 수정본) ---
✅ '영문' 목록 파일(58건) 로드 성공.
총 58건의 영문 'overview' 수집을 시작합니다...


  0%|          | 0/58 [00:00<?, ?it/s]

------------------------------
✅ [3-D 완료] /content/drive/MyDrive/D-up_Fest_compass/중간발표 코랩/data/seoul_festivals_eng_detail_202501-202512.csv 파일 저장 완료.
   contentid                                          title  sigungucode                                               overview
0    3544280  100 Years Market 100 Years Night (백년시장 백년나이트)          3.0  100 Year Market, a traditional market with a long ...
1    3500481             2025 APP WORLD TOUR SEOUL SUP OPEN         18.0  The APP World Tour is a global stand-up paddeboard...
2    2456838          Achasan Sunrise Festival (아차산 해맞이 축제)          6.0  Achasan Mountain offers a great view of the sunris...
3    3523415                                   Almond (아몬드)         23.0                                                    ...
4    3529576                         Another Today(또 다른 오늘)         23.0                                                    ...


In [ ]:
df_eng_detail.iloc[0]

,0
contentid,3544280
title,100 Years Market 100 Years Night (백년시장 백년나이트)
sigungucode,3.0
overview,"100 Year Market, a traditional market with a l..."


# 국문 일반/상세 정보 통합

In [ ]:
import pandas as pd
import os

print("--- [Step 3-E] '국문' 일반/상세 정보 통합 시작 ---")

# --- 1. 경로 설정 (기존 셀들에서 정의된 변수 활용) ---
# BASE_PATH = "/content/drive/MyDrive/D-up_Fest_compass/중간발표 코랩/data/"
# PERIOD = "202501-202512"

INPUT_KOR_CSV_PATH = os.path.join(BASE_PATH, "seoul_festivals_kor.csv")
INPUT_KOR_DETAIL_PATH = os.path.join(BASE_PATH, f"seoul_festivals_kor_detail_{PERIOD}.csv")
OUTPUT_KOR_TOTAL_PATH = os.path.join(BASE_PATH, "seoul_festivals_kor_total.csv")

try:
    # --- 2. 파일 로드 ---
    # 2-1. 일반 정보 로드 (seoul_festivals_kor.csv)
    df_kor_base = pd.read_csv(INPUT_KOR_CSV_PATH)
    print(f"✅ 원본 일반 정보 로드 완료: {df_kor_base.shape}")

    # 2-2. 상세 정보 로드 (overview 컬럼만 추출)
    df_kor_detail = pd.read_csv(INPUT_KOR_DETAIL_PATH)
    # overview 정보만 필요하므로, 중복되는 'title', 'sigungucode'는 제외하고 'contentid'와 'overview'만 선택
    df_overview_only = df_kor_detail[['contentid', 'overview']]
    print(f"✅ 상세 설명(overview) 정보 로드 완료: {df_overview_only.shape}")

    # --- 3. 데이터 병합 (Left Merge) ---
    # df_kor_base(왼쪽)를 기준으로 df_overview_only(오른쪽)의 'overview' 컬럼을 'contentid'를 키로 하여 붙입니다.
    df_total = pd.merge(
        df_kor_base,
        df_overview_only,
        on='contentid',
        how='left'  # 원본(df_kor_base)의 모든 행을 유지
    )

    # --- 4. 최종 파일 저장 ---
    df_total.to_csv(OUTPUT_KOR_TOTAL_PATH, index=False, encoding='utf-8-sig')

    print("---" * 10)
    print(f"✅ [3-E 완료] 최종 통합 파일 저장 완료: {OUTPUT_KOR_TOTAL_PATH}")
    print(f"   -> 최종 데이터 크기: {df_total.shape}")
    print("\n[병합된 데이터 샘플 (마지막 'overview' 컬럼 확인)]")
    # 'overview' 컬럼이 잘 추가되었는지 확인
    print(df_total[['contentid', 'title', 'sigungucode', 'overview']].head())

except FileNotFoundError as e:
    print(f"❌ [에러] 파일 로드 실패. '{e.filename}' 파일을 찾을 수 없습니다.")
    print("   -> [Step 3-A]와 [Step 3-C]가 먼저 실행되었는지 확인하세요.")
except Exception as e:
    print(f"❌ [에러] 데이터 병합 중 알 수 없는 오류 발생: {e}")

--- [Step 3-E] '국문' 일반/상세 정보 통합 시작 ---
✅ 원본 일반 정보 로드 완료: (239, 29)
✅ 상세 설명(overview) 정보 로드 완료: (239, 2)
------------------------------
✅ [3-E 완료] 최종 통합 파일 저장 완료: /content/drive/MyDrive/D-up_Fest_compass/중간발표 코랩/data/seoul_festivals_kor_total.csv
   -> 최종 데이터 크기: (239, 30)

[병합된 데이터 샘플 (마지막 'overview' 컬럼 확인)]
   contentid           title  sigungucode  \
0    3113671  가락몰 빵축제 전국빵지자랑           18   
1    3379778   가락옥토버페스트 미식야행           18   
2    3021762       가을 , 명동으로           24   
3    3541003          강남청년축제            1   
4    1806376         강동북페스티벌            2   

                                            overview  
0  전국 각지의 농수축산물이 모이는 가락몰에서, 전국 각지의 빵 맛집들이 모여 서울 최...  
1  전국 각지 농수산물이 모이는 서울시 송파구 가락몰 3층 하늘공원에서 루프탑 야외축제...  
2  '가을, 명동으로'는 즐거움과 의미가 함께하는 행사이다. 명동역 3번 출구 앞 ‘재...  
3  2025 강남청년축제 ‘소셜링 페스티벌’은 강남구를 대표하는 청년축제이다. 축제는 ...  
4  강동북페스티벌은 강동문화재단이 주최하고 강동구립도서관이 주관하는 독서문화축제로서, ...  


In [7]:
df_total = pd.read_csv("/content/drive/MyDrive/D-up_Fest_compass/중간발표 코랩/data/seoul_festivals_kor_total.csv")

In [8]:
print(df_total.iloc[0])

addr1                                      서울특별시 송파구 양재대로 932 (가락동)
addr2                                                   가락몰 3층 하늘공원
zipcode                                                      5699.0
cat1                                                            A02
cat2                                                          A0207
cat3                                                      A02070200
contentid                                                   3113671
contenttypeid                                                    15
createdtime                                          20240415100726
eventstartdate                                             20250509
eventenddate                                               20250511
firstimage        http://tong.visitkorea.or.kr/cms/resource/91/3...
firstimage2       http://tong.visitkorea.or.kr/cms/resource/91/3...
cpyrhtDivCd                                                   Type3
mapx                                            